In [10]:
from plot_data_utilities import plot_config_map, plot_visits_map
import json
import os, pandas as pd, matplotlib.pyplot as plt, numpy as np
from matplotlib.lines import Line2D
from tabulate import tabulate


In [11]:
curr_folder = os.getcwd()
main_dir = os.path.abspath(os.path.join(curr_folder, os.pardir))
input_dir_raw = os.path.join(main_dir, 'data_raw')

In [12]:
def find_key(nested_data, target_key):
    if isinstance(nested_data, list):
        for data in nested_data:
            result = find_key(data, target_key)
            if result is not None:
                return result
    elif isinstance(nested_data, dict):
        for key, value in nested_data.items():
            if key == target_key:
                return value
            if isinstance(value, (dict, list)):
                result = find_key(value, target_key)
                if result is not None:
                    return result
    return None

In [13]:
# Extract all configuration data

for folder in os.listdir(curr_folder):
    if os.path.isdir(os.path.join(curr_folder, folder)) and folder[:2].isdigit() and int(folder[:2]) >= 2:
        print(folder)
        subfolder = os.path.join(curr_folder, folder)
        
        for folder in os.listdir(subfolder):
            if os.path.isdir(os.path.join(subfolder, folder)) and not folder == "__pycache__":
                exp_folder = os.path.join(subfolder, folder)
                if not "policy_data.csv" in os.listdir(exp_folder):
                    continue

                #print(folder)
                exp_name = subfolder.split('/')[-1]
                result_dir = os.path.join(main_dir, "results/", exp_name, folder)
                
                #print(result_dir)

                if not os.path.exists(result_dir):
                    os.makedirs(result_dir)

                # Load the JSON file
                with open(os.path.join(input_dir_raw, exp_name, folder, "0", "conf_dict.json")) as f:
                    conf_dict = json.load(f)

                
                agent_conf_0 = conf_dict['agent_conf_0']
                agent_conf_1 = conf_dict['agent_conf_1']
                
                # Create a list of tuples for the table
                table_conf_data = []
                for key in agent_conf_0['0']:
                    value_0 = agent_conf_0['0'][key]
                    value_1 = agent_conf_1['0'][key]
                    if isinstance(value_0, list):
                        value_0 = [round(val, 2) for val in value_0]
                    if isinstance(value_1, list):
                        value_1 = [round(val, 2) for val in value_1]
                    table_conf_data.append(('0', key, value_0, value_1))

                for key in agent_conf_1['1']:
                    value_1 = agent_conf_1['1'][key]
                    value_0 = agent_conf_0['1'][key]
                    if isinstance(value_0, list):
                        value_0 = [round(val, 2) for val in value_0]
                    if isinstance(value_1, list):
                        value_1 = [round(val, 2) for val in value_1]
                    table_conf_data.append(('1', key, value_1, value_0))
                    

                # Create the table using tabulate
                table_conf_0 = tabulate(table_conf_data, headers=['Agent', 'Parameter', 'Values', 'Values assumed by Opponent'], tablefmt='latex')
                table_conf_0 = "\\begin{table}[H]\n\\centering\n" + table_conf_0
                table_conf_0 += "\n\\caption{Action Configurations}\n\\label{tab:actionassump_"+exp_name+"_"+folder+"}\n\\end{table}"
               

                
                algo_conf_0 = conf_dict['algo_conf_0']
                algo_conf_1 = conf_dict['algo_conf_1']

                algo_keys = ['num_iter', 'discount_factor', 'weight_interm', 'weight_final', 'feature_flags']
                
                table_algo =[]
                for keys in algo_keys:
                    try:
                        value_0_real = find_key(algo_conf_0["0"], keys)
                        value_0_assumed = find_key(algo_conf_1["0"], keys)
                        value_1_real = find_key(algo_conf_1["1"], keys)
                        value_1_assumed = find_key(algo_conf_0["1"], keys)

                        if keys == 'feature_flags':
                            for feature in value_0_real.keys():
                                flags_0_real = value_0_real[feature]
                                flag_0_real = [flag for flag, v in flags_0_real.items() if v is True][0]

                                flags_0_assumed = value_0_assumed[feature]
                                flag_0_assumed = [flag for flag, v in flags_0_assumed.items() if v is True][0]
                                
                                table_algo.append(("0", feature, flag_0_real, flag_0_assumed))
                            for feature in value_1_real.keys():
                                flags_1_real = value_1_real[feature]
                                flag_1_real = [flag for flag, v in flags_1_real.items() if v is True][0]

                                flags_1_assumed = value_1_assumed[feature]
                                flag_1_assumed = [flag for flag, v in flags_1_assumed.items() if v is True][0]
  
                            continue
                        table_algo.append(("0", keys, value_0_real, value_0_assumed))
                    except:
                        value_0_real = find_key(algo_conf_0, keys)
                        value_0_assumed = find_key(algo_conf_0, keys)
                        value_1_real = find_key(algo_conf_1, keys)
                        value_1_assumed = find_key(algo_conf_1, keys)

                        if keys == 'feature_flags':
                            for feature in value_0_real.keys():
                                flags_0_real = value_0_real[feature]
                                flag_0_real = [flag for flag, v in flags_0_real.items() if v is True][0]

                                flags_0_assumed = value_0_assumed[feature]
                                flag_0_assumed = [flag for flag, v in flags_0_assumed.items() if v is True][0]
                                
                                table_algo.append(("0", feature, flag_0_real, flag_0_assumed))
                            for feature in value_1_real.keys():
                                flags_1_real = value_1_real[feature]
                                flag_1_real = [flag for flag, v in flags_1_real.items() if v is True][0]

                                flags_1_assumed = value_1_assumed[feature]
                                flag_1_assumed = [flag for flag, v in flags_1_assumed.items() if v is True][0]
 
                            continue
                        table_algo.append(("0", keys, value_0_real, value_0_assumed))

                for keys in algo_keys:
                    try:
                        value_0_real = find_key(algo_conf_0["0"], keys)
                        value_0_assumed = find_key(algo_conf_1["0"], keys)
                        value_1_real = find_key(algo_conf_1["1"], keys)
                        value_1_assumed = find_key(algo_conf_0["1"], keys)

                        if keys == 'feature_flags':
                            for feature in value_0_real.keys():
                                flags_0_real = value_0_real[feature]
                                flag_0_real = [flag for flag, v in flags_0_real.items() if v is True][0]

                                flags_0_assumed = value_0_assumed[feature]
                                flag_0_assumed = [flag for flag, v in flags_0_assumed.items() if v is True][0]
                                
                            for feature in value_1_real.keys():
                                flags_1_real = value_1_real[feature]
                                flag_1_real = [flag for flag, v in flags_1_real.items() if v is True][0]

                                flags_1_assumed = value_1_assumed[feature]
                                flag_1_assumed = [flag for flag, v in flags_1_assumed.items() if v is True][0]

                                table_algo.append(("1", feature, flag_1_real, flag_1_assumed))  
                            continue
                        table_algo.append(("1", keys, value_1_real, value_1_assumed))
                    except:
                        value_0_real = find_key(algo_conf_0, keys)
                        value_0_assumed = find_key(algo_conf_0, keys)
                        value_1_real = find_key(algo_conf_1, keys)
                        value_1_assumed = find_key(algo_conf_1, keys)

                        if keys == 'feature_flags':
                            for feature in value_0_real.keys():
                                flags_0_real = value_0_real[feature]
                                flag_0_real = [flag for flag, v in flags_0_real.items() if v is True][0]

                                flags_0_assumed = value_0_assumed[feature]
                                flag_0_assumed = [flag for flag, v in flags_0_assumed.items() if v is True][0]
                                
                            for feature in value_1_real.keys():
                                flags_1_real = value_1_real[feature]
                                flag_1_real = [flag for flag, v in flags_1_real.items() if v is True][0]

                                flags_1_assumed = value_1_assumed[feature]
                                flag_1_assumed = [flag for flag, v in flags_1_assumed.items() if v is True][0]

                                table_algo.append(("1", feature, flag_1_real, flag_1_assumed))  
                            continue
                        table_algo.append(("1", keys, value_1_real, value_1_assumed))
                
                #print('table_algo_0_final', table_algo_0)
                table_algo_cleaned = tabulate(table_algo, headers=['Agent', 'Parameter', 'Values', 'Values assumed by Opponent'], tablefmt='latex')
                table_algo_cleaned = "\\begin{table}[H]\n\\centering\n" + table_algo_cleaned
                table_algo_cleaned += "\n\\caption{Algorithmic Configurations}\n\\label{tab:algoassump_"+exp_name+"_"+folder+"}\n\\end{table}"

                
                

                config_file = os.path.join(result_dir, "config.tex")
                with open(config_file, 'w') as f:
                    f.write('\n\n')
                    f.write(table_conf_0)
                    f.write('\n\n')
                    f.write(table_algo_cleaned)



                

04_EE-NN
03_EE-EE-agentdiff
07_EE-EE_prioritize
02_EE-EE
03_EE-EE-veldiff
02_EE-EE_20k
04_NN-NN


In [72]:
# obtain number of collisions and lead at final timestep for each agent

